## Notebook to process GNSS data for each receiver

### Step 1: Load libraries

Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 

In [2]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

### Step 2: Time intervals

Set up time intervals for the study period. The argument **periods** represents the number of days from the start day.

In [4]:
startday = pd.to_datetime('01-03-2025',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-03-01 00:00:00, 2025-03-02 00:00:00),
               [2025-03-02 00:00:00, 2025-03-03 00:00:00),
               [2025-03-03 00:00:00, 2025-03-04 00:00:00),
               [2025-03-04 00:00:00, 2025-03-05 00:00:00),
               [2025-03-05 00:00:00, 2025-03-06 00:00:00),
               [2025-03-06 00:00:00, 2025-03-07 00:00:00),
               [2025-03-07 00:00:00, 2025-03-08 00:00:00),
               [2025-03-08 00:00:00, 2025-03-09 00:00:00),
               [2025-03-09 00:00:00, 2025-03-10 00:00:00),
               [2025-03-10 00:00:00, 2025-03-11 00:00:00)],
              dtype='interval[datetime64[ns], left]')

### Step 3: Read RINEX file

We then proceed to read RINEX file. This recquires to indicate the location of the file (**pattern**) and the location of the output directory (**outputdir**). Sometimes we could get the following error:

ValueError: Missing an approximate antenna position. Provide the argument ‘approx_position’ to preprocess()

In [11]:
pattern= {'YoungPine-1':'/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_tower/Reach01_raw_20250331194333.25O'}
outputdir={'YoungPine-1':'/Users/coopercollins/box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_tower/youngpine_tower_nc/'}
#approx_position=[-4705.036,43.000,23011766.990]
#keepvars=['S1C','S1X','S2C','S2X']
#gv.preprocess(pattern,interval='30s',keepvars=keepvars,outputdir=outputdir, approx_position=approx_position)
result = gv.preprocess(pattern,outputresult=True)


Created a temporary directory at /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh
/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_tower/Reach01_raw_20250331194333.25O exists | Reading...
Observation file  /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_tower/Reach01_raw_20250331194333.25O  is read in 41.79 seconds.
Processing 3337663 individual observations
Calculating Azimuth and Elevation
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250900000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2360//GFZ0MGXRAP_20250900000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250900000_01D_05M_ORB.SP3.gz: 1.05MB [00:03, 363kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2360//GFZ0MGXRAP_20250900000_01D_05M_ORB.SP3.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250900000_01D_05M_ORB.SP3 file is read in 3.35 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250910000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2360//GFZ0MGXRAP_20250910000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250910000_01D_05M_ORB.SP3.gz: 1.05MB [00:02, 425kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2360//GFZ0MGXRAP_20250910000_01D_05M_ORB.SP3.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250910000_01D_05M_ORB.SP3 file is read in 2.88 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250900000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250900000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20250900000_01D_30S_CLK.CLK.gz: 4.79MB [00:05, 986kB/s]                             


 | Download completed for GFZ0MGXRAP_20250900000_01D_30S_CLK.CLK.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250900000_01D_30S_CLK.CLK file is read in 1.10 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250910000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250910000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20250910000_01D_30S_CLK.CLK.gz: 4.79MB [00:04, 1.09MB/s]                            


 | Download completed for GFZ0MGXRAP_20250910000_01D_30S_CLK.CLK.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh/GFZ0MGXRAP_20250910000_01D_30S_CLK.CLK file is read in 1.16 seconds
SP3 interpolation is done in 11.75 seconds
Removed the temporary directory at /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmptdl5tilh


In [12]:
obs = result['YoungPine-1'][0]
obs.observation

C1C  C1X  C2C           C2I           C2X  \
Epoch               SV                                                        
2025-03-31 19:43:54 G06  1.963710e+07  NaN  NaN           NaN  1.963710e+07   
                    G11  1.928477e+07  NaN  NaN           NaN  1.928477e+07   
                    G12  1.893834e+07  NaN  NaN           NaN  1.893835e+07   
                    G17  2.405567e+07  NaN  NaN           NaN  2.405569e+07   
                    G19  2.099305e+07  NaN  NaN           NaN           NaN   
...                               ...  ...  ...           ...           ...   
2025-04-01 19:43:53 C24           NaN  NaN  NaN  2.180327e+07           NaN   
                    C25           NaN  NaN  NaN  2.253623e+07           NaN   
                    C26           NaN  NaN  NaN  2.535395e+07           NaN   
                    C34           NaN  NaN  NaN  2.236150e+07           NaN   
                    C35           NaN  NaN  NaN  2.453486e+07           NaN   

                         C7I  C7X       D1C  D1X  D2C  ...  L7X   S1C  S1X  \
Epoch               SV                                 ...                   
2025-03-31 19:43:54 G06  NaN  NaN -1395.467  NaN  NaN  ...  NaN  48.0  NaN   
                    G11  NaN  NaN  1037.604  NaN  NaN  ...  NaN  47.0  NaN   
                    G12  NaN  NaN  1273.991  NaN  NaN  ...  NaN  48.0  NaN   
                    G17  NaN  NaN -2754.732  NaN  NaN  ...  NaN  34.0  NaN   
                    G19  NaN  NaN -2636.434  NaN  NaN  ...  NaN  43.0  NaN   
...                      ...  ...       ...  ...  ...  ...  ...   ...  ...   
2025-04-01 19:43:53 C24  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C25  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C26  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C34  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   
                    C35  NaN  NaN       NaN  NaN  NaN  ...  NaN   NaN  NaN   

                         S2C   S2I   S2X  S7I  S7X     Azimuth  Elevation  
Epoch               SV                                                     
2025-03-31 19:43:54 G06  NaN   NaN  42.0  NaN  NaN   62.949178  56.526398  
                    G11  NaN   NaN  44.0  NaN  NaN  144.182651  69.135951  
                    G12  NaN   NaN  42.0  NaN  NaN  -49.081408  70.360939  
                    G17  NaN   NaN  28.0  NaN  NaN   72.673754   3.773296  
                    G19  NaN   NaN   NaN  NaN  NaN   65.809338  28.837489  
...                      ...   ...   ...  ...  ...         ...        ...  
2025-04-01 19:43:53 C24  NaN  49.0   NaN  NaN  NaN -170.511731  60.548053  
                    C25  NaN  47.0   NaN  NaN  NaN  -60.240996  46.413606  
                    C26  NaN  40.0   NaN  NaN  NaN  156.256091  16.535907  
                    C34  NaN  47.0   NaN  NaN  NaN -118.839301  49.017806  
                    C35  NaN  44.0   NaN  NaN  NaN   56.427768  23.414502  

[3337663 rows x 30 columns]